# 19f: further ideas to further improve your setup

In this notebook we gathered all hints and further optional ideas. No obligation to use them, but conveniently presented here so you can cherrypick any setup improvement you would like to explore.

## Relay
### Relay in case of DAC A + DAC Assistant

The electrochemical reaction within the measurement cell continues as long as a current can flow. To prevent undesired reactions during idle periods, it's crucial to make sure that the signals applied to the cell do not generate a substantial current.

💡 Hint: what happens to Ucell when DAC A and DAC assistant are both used, and DAC A is set to zero?

This can be achieved by setting the voltage to the cell to zero, by controlling the voltage in the potentiometric regulation loop. This comes with its own challenges in this design, which you will discover during the test procedures.

Alternatively, the relay on the ALPACA board can be used to electrically disconnect the measurement cell from the control setup. The relay is located on the ALPACA board, close to the construction site. It can be controlled using the pin labelled `Relay` on `J8`. The relay will react (energize the electro-magnet) when a signal 3.3V or 5V is applied to the pin “relay”.

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/diconnect_via_relay.JPG" width=400><br>
    <em>ALPACA's Relay</em>
</div><br>

>In its **default state**, the electromagnet is not energized:
>- the relay connects the `COMMON` terminal to the `NORMAL CLOSED`. <br>
>
>In its **energized state**, the electromagnet switches this connection:
>- and the relay links the `COMMON` terminal with `NORMAL OPEN`, effectively connecting the measurement cell.
>
>
>
> Use the `GPIO14` pin of `J1` on Cria to control the action of the relay. <br>
> You can instantiate and manipulate the Python object for the relay with:
> ```python
> from machine import Pin
> relay = Pin(14, Pin.OUT)
> relay.value(0) ## OFF
> relay.value(1) ## ON
> ``` 

Have a look into the Alpaca Manual (Download from Brightspace) for more info on the use of the relay and **a complete Alpaca circuit layout on the last page of the manual.**

### Relay in case of switching amplification in one of the Amplifiers
Add the relay and its control to the circuit for switching between `Rf` options. 

## Using AMP1 as an inverting attenuator 

If you have a look into the Alpaca's circuit diagram in the Alpaca Manual, the last page, and find the circuit of the amplifier associated with `ADC1` (*Hint: Look for jumpers and resistors in series *60*, i.e. `J62` or `R69` to find `AMP1`*), you will notice that the `J62:ATTENUATOR` only works for signals measured via `J60:SIGNAL+`.

Reading and understanding Alpaca's circuit diagram may seem daunting and complicated at first, but it turns out that in order to attenuate a negative input and match the attenuation on the positive part of the signal, you will simply have to add an extra resistor before the inverting input of the OPAMP.  
 
The default setting for `J60:SIGNAL-` is `GAIN=-1` and it is realised with the following combination:
1. `Ua` to `J60:SIGNAL- = J60:3` 
2. `J60:SIGNAL+ = J60:2` to `J60:GND = J60:1`
3. `J60:DC`
5. `J62:1-4 = J62:B = 0 Ohm`
6. `J63:1-4 = J63:B = 0 Ohm`
7. `J65:1-4 = J65:B = 0 Ohm`
8. No jumpers on `J64` and `J17`


## Limits of the ADCs: offset and noise

Did you know that measuring very low voltages with Pico might be problematic? This is especially relevant for the Double ADC Design. Can you argue why it is so?

>Goal 1: Measure the baseline noise level, without any signal on the ADC's. <br>
>Goal 2: Observe ADC ground error and the improvement with `-12V` connected to Cria

<details>
<summary>
<font color='darkred' size=4>💡<b>Fritzing: ADC Test</b></font>
</summary>

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_Alpaca_Check_Jumpers.png" width=1000>
<br>
<em>ADC Baseline Test - Do not connect anything to ADCs</em>
</div><br>
</details> 
<br>

### I2.1: Baseline offset

First, find out what's the baseline offset of the ADCs, so without any inputs 

> Behind the scenes, the `ADC0` and `ADC1` will be measured via the Alpaca's amplifiers. Let's see if they're really 0V!
> In detail:
>	1. `Ain0` via `ADC0` (Jumpers on AMPLIFIER DIRECT TO NANO, directly in the cut on the right side of the Cria)
>	2. `Ain1` via `ADC1`
>	3. `Ain2` directly

Let's start with *the reset* to develop a habit:

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Run the test:
AMP0, AMP1, Ain2 = pico.test_ADC()

In [ ]:
# Remember to convert ADC samples to Volts!
AMP0v = pico.convert_samples_to_volts(AMP0, gain=1)
AMP1v = pico.convert_samples_to_volts(AMP1, gain=1)
Ain2v = pico.convert_samples_to_volts(Ain2, gain=1)

# Plot the baseline offset
plt.plot(AMP0v, label='AMP0')
plt.plot(AMP1v, label='AMP1')
plt.plot(Ain2v, label='Ain2')
plt.xlabel('Sample')
plt.ylabel('Signal [V]')
plt.legend()

It's most likely not 0V!

Let's run some statistics and investigate this further.

### ADC - Noise

Run the cells below to compute the errors and find out more about the nature of this noise.

In [ ]:
avg_signal_Ain0, std_dev_Ain0 = pico.compute_noise_statistics(AMP0v)
avg_signal_Ain1, std_dev_Ain1 = pico.compute_noise_statistics(AMP1v)
avg_signal_Ain2, std_dev_Ain2 = pico.compute_noise_statistics(Ain2v)



1. Make a note of the magnitude of the error. **Will it affect your Voltammetry measurements?**
2. What is the magnitude of the noise? **How does it compare to the resolution of ADCs?**

In [ ]:
### Notes

In [ ]:
pico.plot_noise_spectrum(AMP0v, label="AMP0")
pico.plot_noise_spectrum(AMP1v, label="AMP1")
pico.plot_noise_spectrum(Ain2v, label="Ain2")
plt.legend()

3. Are there any significantly dominanting frequencies?

Let's save the original data for practice, and for reference!

In [ ]:
DATA=np.zeros((3,pico.NUM_SAMPLES))
DATA[0,:]=AMP0 
DATA[1,:]=AMP1
DATA[2,:]=Ain2

np.save('temporary_data.npy', DATA)
del(DATA)

In [ ]:
%fetchfile --binary "temporary_data.npy" "ADC_Baseline_Test_DATA.npy" # for ADC-noise
#%fetchfile --binary "temporary_data.npy" "ADC_Baseline_Test_DATA-12V.npy" #for I2.3 (the next section)

### Test ADCs with -12V connected to Cria

<font color='#FF5F15' size=4>⚠️</font> 
<font color='#FF5F15' size=3><b>Warning:</b> Use the Fritzing below to carefully connect -12V to Cria: Orange LED will light up!</font> 
<details>
<summary>
<font size=3>💡</font> <b>Fritzing: -12V to Cria</b>
</summary>

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_Alpaca_ADC_test_with_-12in.png" width=1000>
<br>
<em>Connecting -12V source in Alpaca to Cria's J5:-12V in </em>
</div><br>
</details> 


1. Re-run the test from *I2.2* with `-12V` connected to the '-12V in' pin on the multifunction connector. 
2. Use a different name for your data file on your laptop (ADC_Baseline_Test_DATA), otherwise you overwrite previous data. Use our hint in the comments.
3. In the cell below, compare the average noise signal without any input signal, in the two cases: with and without `-12V in`
4. Argue whether it is better to work with `-12V in` or without?

In [ ]:
### Notes


## DAC - Noise

Run the cells below to measure the noise of the DAC output.

In [ ]:
DCsetpoint = 1 # Set the DACA output value to test its accuracy
AMP0,AMP1,Ain2 = pico.SetDAC_and_MeasureADC0andADC1andADC2(DCsetpoint)

In [ ]:
AMP0v = pico.convert_samples_to_volts(AMP0)
AMP1v = pico.convert_samples_to_volts(AMP1, gain=0.333)
Ain2v = pico.convert_samples_to_volts(Ain2)

In [ ]:
plt.plot(AMP0v[:500], label='AMP0')
plt.plot(AMP1v[:500], label='AMP1')
plt.plot(Ain2v[:500], label='Ain2')
plt.xlabel('Sample')
plt.ylabel('Signal [V]')
plt.legend()

It's probably not as stable as you would expect it!

Let's run some familiar statistics in the next section.

In [ ]:
avg_signal_AMP0v, std_dev_AMP0v = pico.compute_noise_statistics(AMP0v, DC=DCsetpoint)
avg_signal_AMP1v, std_dev_AMP1v = pico.compute_noise_statistics(AMP1v, DC=DCsetpoint)
avg_signal_Ain2v, std_dev_Ain2v = pico.compute_noise_statistics(Ain2v, DC=DCsetpoint)

In [ ]:
pico.plot_noise_spectrum(AMP0v, label="AMP0")
pico.plot_noise_spectrum(AMP1v, label="AMP1")
pico.plot_noise_spectrum(Ain2v, label="Ain2")
plt.legend()

### Conclude 

Just like in the ADC Test:

1. Make a note of the magnitude of the error. **Will it affect your Voltammetry measurements?**
2. Is there any significantly dominant frequency? **Does it change when you disconnect -12V from Cria?**
3. What is the magnitude of the noise? **How does it compare to the resolution of DAC?**

The last answer is especially important for finding out the limits for the Voltammetry measurements.

In [ ]:
### Notes

Let's save the acquired data - *for practice, and for future reference!*

In [ ]:
DATA=np.zeros((3,pico.NUM_SAMPLES))
DATA[0,:]=Ain0v 
DATA[1,:]=Ain1v
DATA[2,:]=Ain2v

np.save('temporary_data.npy', DATA)
del(DATA)

In [ ]:
%fetchfile --binary "temporary_data.npy" "DAC_Setpoint_Test_DATA_-12V.npy"
# %fetchfile --binary "temporary_data.npy" "DAC_Setpoint_Test_DATA.npy"

## Implement 4: Timing accuracy


The function provided below `pico.test_pico_timing_with_for_loop()` measures the time between each step of a measurement with averaging.
You can adjust two parameters:

1. `N_iter` sets the number of measurements for all three ADCs that are then averaged at each step of your experiment 
2. `delay_ms` sets the delay between each step of your experiment

By default, `NUM_SAMPLES=512` for demonstration.

Run the code below for different values, for example:

- `N_iter=1,3,10`
- `delay_ms=1,2,5,10`

Make notes from your observations, and feel free to reach out to the TAs for a discussion.

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt
import machine

In [ ]:
times = pico.test_pico_timing_with_for_loop(N_iter=1,delay_ms=1)

In [ ]:
plt.plot(times*1e-3)
plt.xlabel("Sample")
plt.ylabel("Duration of the measurement [ms]")
plt.grid(True)

In [ ]:
### Notes

<details>
<summary><font size=4>💡</font> <b>Hints </b>
</summary>

Note that `pico.test_pico_timing_with_for_loop()` uses a `for` loop for averaging over multiple measurements. It has an `if`-statement, and the averaging happens between the steps. Also, note that *time-keeping* takes some time too, so there are many operations additional to setting the DAC value and reading the ADCs.<br>
The results above demonstrate how essential it is to optimise your code for the final measurements, especially if you are using averaging and DELAY_MS &lt; 10ms" 
</details>

## Double ADC Design: Noise 

**This section is optional, but it is recommended! You can learn more about the magnitude of noise in the Double ADC Design to be able to identify problems later in your design**

Use the same test setup as for *Implement 5B.1: Calibration*, follow the steps, and write down your conclusions.
> This test takes a few seconds
> 
> `NUM_SAMPLES` sets the number of points across the test range , which is -3V to 3V by default <br>
> `interations` sets the number of samples at each step to computer the noise statistics <br>
>
> For large values, you might expect problems with Pico's memory and plotting.

In [ ]:
machine.soft_reset()

In [ ]:
%serialconnect to --port="COM5"

%sendtofile /picotools.py --source picotools.py

import picotools as pico

import numpy as np
import matplotlib.pyplot as plt
import machine

In [ ]:
calibrated_offset = pico.Calibrate_DAC_Assistant(1.5)

In [ ]:
ref, Ain0, Ain1, Ain2, Ain0std, Ain1std, Ain2std, err_in, err_out = pico.test_DAC_Assistant_noise(offset=calibrated_offset, NUM_SAMPLES=600, iterations = 100, gain0=1, gain1=-1)

In [ ]:
plt.plot((5 * (ref - calibrated_offset)), Ain0std*1e3, label='Noise via ADC0')
plt.plot((5 * (ref - calibrated_offset)), -Ain1std*1e3, label='Noise via ADC1')
plt.xlabel("DAC Assistant output [V]")
plt.ylabel("ADC average readout error [mV]")
plt.legend()

In [ ]:
### Notes

##Double ADC design Noise - (GAIN 1:3) - Optional

**This section is entirely optional.** Use it for debugging and to understand the cumulative effect of noise due to amplification and attenuation. 
> There is an interesting difference between the results of GAIN 1:1 (in section I4.3) in and GAIN 1:3 (here). Can you argue what's behind that?

In [ ]:
# GAIN (1:3)
ref, Ain0, Ain1, Ain2, Ain0std, Ain1std, Ain2std, err_in, err_out = pico.test_DAC_Assistant_noise(offset=calibrated_offset, NUM_SAMPLES=240, iterations = 50, gain0=0.3333, gain1=-0.3333)

In [ ]:
plt.plot((5 * (ref - calibrated_offset)), Ain0std*1e3, label='Noise via ADC0')
plt.plot((5 * (ref - calibrated_offset)), -Ain1std*1e3, label='Noise via ADC1')
plt.xlabel("DAC Assistant output [V]")
plt.ylabel("ADC readout error [mV]")
plt.legend()

## Optimise the range of detectable `Icell` to improve the Signal-to-Noise ratio

**Recall or derive the formula for the range of `Icell` as a function of all relevant parameters in Model 1 for your chosen design.**

>**Goal:** Find out how to configure your Potentiostat to ensure that are making use of the avaialable limits to record the strongest signal within a predefined range of `Icell`. 

1. Compute the range of possible `Icell` as a function `Rf` (and in the Variable Offset Design as a function of `Uoffset`) for roughly the entire range of possible `Ua` with the given configuration of resistors.
   > the range of `Ua` is the range of the possible outputs of `OPAMP2`: <br>
   > Double ADC - positive and negative, <br>
   > Variable Offset - only positive.

2. Select `Rf` to increase the current-to-voltage conversion factor and to utilise *roughly* the entire range of `Ua` without causing clipping on `OPAMP2`, and adjust the attenuation of `AMPs` to capture the signal from `Ua` without *clipping* the `ADCs`.

    > Note that it could be optimal to slightly re-adjust the value of `Rf` because there are only a few available options of attenuation factors. It is also fine to leave some margins. 
 
   **Variable Offset Design:** Focus on optimising the range for the positive currents first, so do not change the `Uoffset` yet. Notice what effect the change of `Rf` has on the `Icell` range for the negative currents.

3. **Variable Offset Design:** Derive a formula for setting a desired range of detectable `Icell` and describe the method to adjust that range while maintaining a fixed range of `Ucell`.
   > Consider writing a function to compute that `Icell` range for a given set of parameters or to compute the required parameters for a given range of currents.

4. Pre-compute several essential `Icell` ranges for a pre-defined set of ranges of `Ucell`.
   > **Optional:** Write a function to compute the required parameters on demand.

### Noise reduction
#### Filtering

Figure out how to estimate the bound for a suitable the cut-off frequency of your filter that will only negligibly affect your Voltammetry measurements.

#### Averaging

Gather ideas on how to write an optimised code for measurements with averaging to reduce its effect on the timing precision.

### Custom functions

It is possible to prepare the desired set of instructions for the `DACs` entirely from scratch, without using `Triangle`, which might be a preferred approach in some use cases.  <Br><Br> **The programming approach is then entirely in your hands.**